# Fraud Detection — Fine-tune RoBERTa (Colab)



This notebook uploads `text_dataset.csv`, performs a stratified train/test split, fine-tunes `roberta-base` for fraud classification, and reports evaluation metrics.

In [ ]:
# Install dependencies (Colab)
!pip -q install -U torch transformers scikit-learn pandas numpy

In [ ]:
import os
import io
import warnings
import numpy as np
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    f1_score,
    accuracy_score,
    average_precision_score,
)

from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)

warnings.filterwarnings("ignore")

try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [ ]:
# Config

OUTPUT_DIR = Path('./roberta_fraud_model')

MODEL_NAME = 'roberta-base'

MAX_LEN = 256

BATCH_SIZE = 8

GRAD_ACCUM = 4

EPOCHS = 5

LR = 2e-5

SEED = 42



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using device: {device}')

if device.type == 'cuda':

    print(f'GPU: {torch.cuda.get_device_name(0)}')



print('\nNext cell will ask you to upload text_dataset.csv')

In [ ]:
# Upload and load text_dataset.csv

if IN_COLAB:

    print('Upload text_dataset.csv ...')

    uploaded = files.upload()

    if 'text_dataset.csv' not in uploaded:

        raise ValueError(f'Expected text_dataset.csv. Uploaded: {list(uploaded.keys())}')

    df = pd.read_csv(io.BytesIO(uploaded['text_dataset.csv']))

else:

    df = pd.read_csv('processed_data/text_dataset.csv')



text_cols = ['title_cleaned', 'description_cleaned', 'review1_cleaned', 'review2_cleaned']

for col in text_cols:

    df[col] = df[col].fillna('')



df['text'] = (

    'Title: ' + df['title_cleaned']

    + ' Description: ' + df['description_cleaned']

    + ' Review1: ' + df['review1_cleaned']

    + ' Review2: ' + df['review2_cleaned']

)



print(f'Dataset shape: {df.shape}')

print(f'Fraud distribution:\n{df["fraud_label"].value_counts()}\n')

In [ ]:
# Train / Test split (stratified)
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df['fraud_label']
)
print(f'Train: {len(train_df)} | Test: {len(test_df)}')
print(f'Train fraud ratio: {train_df["fraud_label"].mean():.4f}')
print(f'Test  fraud ratio: {test_df["fraud_label"].mean():.4f}\n')

In [ ]:
# Tokenizer & dataset
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

class FraudTextDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=MAX_LEN,
            return_tensors='pt',
        )
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx],
        }

print('Tokenizing training set ...')
train_dataset = FraudTextDataset(train_df['text'].tolist(), train_df['fraud_label'].tolist())
print('Tokenizing test set ...')
test_dataset = FraudTextDataset(test_df['text'].tolist(), test_df['fraud_label'].tolist())

In [ ]:
# Class weights (handle imbalance)
n_neg = (train_df['fraud_label'] == 0).sum()
n_pos = (train_df['fraud_label'] == 1).sum()
weight_for_0 = 1.0
weight_for_1 = n_neg / n_pos
class_weights = torch.tensor([weight_for_0, weight_for_1], dtype=torch.float).to(device)
print(f'Class weights: not-fraud={weight_for_0:.2f}, fraud={weight_for_1:.2f}')

model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop('labels')
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    weight_decay=0.01,
    warmup_ratio=0.1,
    fp16=torch.cuda.is_available(),
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    save_total_limit=2,
    seed=SEED,
    report_to='none',
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
    preds = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds),
        'roc_auc': roc_auc_score(labels, probs),
        'avg_precision': average_precision_score(labels, probs),
    }

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

In [ ]:
# Train
print('=' * 60)
print('  Starting RoBERTa fine-tuning')
print('=' * 60)
trainer.train()

In [ ]:
# Evaluate on test set
preds_output = trainer.predict(test_dataset)
logits = preds_output.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
y_pred = np.argmax(logits, axis=-1)
y_true = test_df['fraud_label'].values

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc = roc_auc_score(y_true, probs)
ap = average_precision_score(y_true, probs)

print(f'Accuracy          : {acc:.4f}')
print(f'F1 Score (fraud)  : {f1:.4f}')
print(f'ROC-AUC           : {roc:.4f}')
print(f'Avg Precision (PR): {ap:.4f}\n')
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=['Not Fraud', 'Fraud']))
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))

In [ ]:
# Save best model + tokenizer
save_path = OUTPUT_DIR / 'best_model'
trainer.save_model(str(save_path))
tokenizer.save_pretrained(str(save_path))
print(f'Best model & tokenizer saved to: {save_path.resolve()}')